In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
df.head(10)

In [ ]:
df['item_cnt_day'] = df.item_cnt_day.abs()
df['item_price'] = df.item_price.abs()

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
df = df.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price':'mean','item_cnt_day':'sum'}).reset_index()

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,8))
dataplot = sns.heatmap(df.corr(), cmap="YlGnBu", annot=True)
plt.show()

In [ ]:
df['shop_id'].value_counts()

In [ ]:
test_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
test_data['date_block_num'] = 34
test_data=test_data[['date_block_num','shop_id','item_id']]
test_data.head()

In [ ]:
item_price=dict(df.groupby('item_id')['item_price'].last().reset_index().values)
test_data['item_price']=test_data.item_id.map(item_price)
print(test_data)
print(df)

In [ ]:
test_data['item_price'] = test_data['item_price'].fillna(test_data['item_price'].median())
print(test_data)

In [ ]:
df = df.sample(frac = 1)
print(df)

In [ ]:
from sklearn.model_selection import train_test_split
X = np.array(df.drop(['item_cnt_day'], axis =1 ))
Y = np.array(df.iloc[:,4])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 43 )

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures  
poly_regs= PolynomialFeatures(degree= 4)  
x_poly= poly_regs.fit_transform(X)  
#clf = RandomForestRegressor(n_estimators=50)
clf = LinearRegression()
clf.fit(x_poly, Y)

In [ ]:
poly_pred = clf.predict(poly_regs.fit_transform(test_data))  
print(poly_pred)  
#test_pred = clf.predict(test_data)

In [ ]:
df1 = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
df2 = pd.DataFrame()
df2['ID'] = df1['ID']
df2['item_cnt_month'] = poly_pred
df2['item_cnt_month'] = df2.item_cnt_month.abs()
df2.to_csv('./result.csv', index= False)